In [54]:
from abc import ABC

from gulpio2 import GulpDirectory
from pathlib import Path
from moviepy.editor import ImageSequenceClip, clips_array

from collections import defaultdict

import pickle
import pandas as pd
from pathlib import Path

import torch as t
import torch.nn as nn
import numpy as np
from omegaconf import OmegaConf
from typing import Any, Dict, List, Sequence, Union

from systems import EpicActionRecognitionSystem
from systems import EpicActionRecogintionDataModule

from utils.metrics import compute_metrics
from utils.actions import action_id_from_verb_noun
from scipy.special import softmax

from GPUtil import showUtilization as gpu_usage
from tqdm import tqdm

In [6]:
class FeatureWriter(ABC):
    def append(self, narration_id: str, features: np.ndarray, labels: Dict[str, Any]) -> None:
        raise NotImplementedError()

In [2]:
# SETUP TORCH VARIABLES
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
dtype = t.float

In [61]:
# def generate_model(path: Path):
    # LOAD IN SAVED CHECKPOINT
ckpt = t.load('../models/trn_rgb.ckpt', map_location='cpu')

    # CREATE CONFIG FROM CHECKPOINT
cfg = OmegaConf.create(ckpt['hyper_parameters'])
OmegaConf.set_struct(cfg, False)

    # SET GULP DIRECTORY
cfg.data._root_gulp_dir = '/home/ts/C1-Action-Recognition-TSN-TRN-TSM/datasets/epic/gulp/rgb'

    # CREATE MODEL
model = EpicActionRecognitionSystem(cfg)
model.load_state_dict(ckpt['state_dict'])

data_module = EpicActionRecogintionDataModule(cfg)
#     return model.model.to(device)
data = data_module.test_dataloader()

# cfg.data.train_gulp_dir
for i, (batch_input, batch_labels) in tqdm(
            enumerate(data),
            unit=" video",
            total=len(data),
            dynamic_ncols=True,
        ):
    print(batch_labels)

Multi-Scale Temporal Relation Network Module in use ['8-frame relation', '7-frame relation', '6-frame relation', '5-frame relation', '4-frame relation', '3-frame relation', '2-frame relation']


 17%|█▋        | 1/6 [00:01<00:08,  1.62s/ video]

{'narration_id': ['P01_01_0', 'P01_01_1', 'P01_01_10', 'P01_01_100', 'P01_01_101', 'P01_01_102', 'P01_01_103', 'P01_01_104', 'P01_01_105', 'P01_01_106', 'P01_01_107', 'P01_01_108', 'P01_01_109', 'P01_01_11', 'P01_01_110', 'P01_01_111', 'P01_01_112', 'P01_01_113', 'P01_01_114', 'P01_01_115', 'P01_01_116', 'P01_01_117', 'P01_01_118', 'P01_01_12', 'P01_01_120', 'P01_01_121', 'P01_01_122', 'P01_01_123', 'P01_01_124', 'P01_01_125', 'P01_01_126', 'P01_01_127', 'P01_01_128', 'P01_01_129', 'P01_01_13', 'P01_01_130', 'P01_01_131', 'P01_01_132', 'P01_01_133', 'P01_01_134', 'P01_01_135', 'P01_01_136', 'P01_01_137', 'P01_01_138', 'P01_01_139', 'P01_01_14', 'P01_01_140', 'P01_01_141', 'P01_01_142', 'P01_01_143', 'P01_01_144', 'P01_01_145', 'P01_01_146', 'P01_01_147', 'P01_01_148', 'P01_01_149', 'P01_01_15', 'P01_01_150', 'P01_01_151', 'P01_01_152', 'P01_01_153', 'P01_01_154', 'P01_01_155', 'P01_01_156'], 'participant_id': ['P01', 'P01', 'P01', 'P01', 'P01', 'P01', 'P01', 'P01', 'P01', 'P01', 'P01',

100%|██████████| 6/6 [00:02<00:00,  2.76 video/s]


In [44]:
class PickleFeatureWriter(FeatureWriter):
    
    def __init__(self, pkl_path: Path, features_dim: int):
        self.pkl_path = pkl_path
        self.features_dim = features_dim
        self.narration_ids = []
        self.features = []
        self.labels = []
        
    def append(self, narration_id: str, features: np.ndarray, labels: Dict[str, Any]) -> None:
#         print(features.shape)
        assert features.shape[1] == self.features_dim
        self.narration_ids.append(narration_id)
        self.features.append(features)
        self.labels.append(labels)
        
    def save(self):
        with open(self.pkl_path, 'wb') as f:
            pickle.dump({
                'narration_id': self.narration_ids,
                'features': np.concatenate(self.features),
                'labels': self.labels
            }, f)

In [45]:
class FeatureExtractor:
    """
    Extracts image features from a 2D CNN backbone for every frame in all videos.
    """
    
    def __init__(
        self, 
        backbone_2d: nn.Module, 
        device: t.device, 
        dtype: t.float, 
        frame_batch_size: int = 128
    ):
        self.model = backbone_2d
        self.device = device
        self.dtype = dtype
        self.frame_batch_size = frame_batch_size
    
    def extract(self, dataset: GulpDirectory, feature_writer: FeatureWriter) -> int:
        total_instances = 0
        self.model.eval()
        for i, c in enumerate(tqdm(dataset)):
            if i == 0:
                for batch_input, batch_labels in c:
        
                    batch_input = np.array(batch_input).transpose(0,3,1,2)
                    batch_input = t.tensor(batch_input, device=self.device, dtype=self.dtype)
                    batch_input = batch_input.unsqueeze(0)

                    batch_size, n_frames = batch_input.shape[:2]
                    flattened_batch_input = batch_input.view((-1, *batch_input.shape[2:]))

                    n_chunks = int(np.ceil(len(flattened_batch_input)/128))
                    chunks = t.chunk(flattened_batch_input, n_chunks, dim=0)
                    flatten_batch_features = []
                    for chunk in chunks:
                        chunk = chunk.unsqueeze(0)
                        with t.no_grad():
                            chunk_features = self.model.features(chunk.to(self.device))
                            chunk_features = self.model.new_fc(chunk_features)
                            flatten_batch_features.append(chunk_features.squeeze(0))
                    flatten_batch_features = t.cat(flatten_batch_features, dim=0)
                    batch_features = flatten_batch_features.view((batch_size, 
                                                                  n_frames, 
                                                                  *flatten_batch_features.shape[1:]))

                    total_instances += batch_size
                    self._append(batch_features, batch_labels, batch_size, feature_writer)
        return total_instances

    def _append(self, batch_features, batch_labels, batch_size, feature_writer):
        batch_narration_id = batch_labels['narration_id']
        assert len([batch_narration_id]) == batch_size
        assert len([batch_labels]) == batch_size
        assert len(batch_features) == batch_size
        batch_features = batch_features.squeeze(0).cpu().numpy()
#         print(batch_narration_id, "DONE")
        feature_writer.append(batch_narration_id, batch_features, batch_labels)

In [46]:
rgb_train = GulpDirectory('../datasets/epic/gulp/rgb/rgb_test/')

feature_writer = PickleFeatureWriter('../datasets/epic/features/p01_features.pkl', features_dim=256)
# for i, c in enumerate(rgb_train):
#     if i == 0:
#         for frames, meta in tqdm(c):
#             xd = frames.shape
extractor = FeatureExtractor(generate_model('../models/trn_rgb.ckpt'), device, dtype)
extractor.extract(rgb_train, feature_writer)

0it [00:00, ?it/s]

Multi-Scale Temporal Relation Network Module in use ['8-frame relation', '7-frame relation', '6-frame relation', '5-frame relation', '4-frame relation', '3-frame relation', '2-frame relation']
P01_01_0 DONE
(195, 256)
P01_01_1 DONE
(109, 256)
P01_01_10 DONE
(75, 256)
P01_01_100 DONE
(68, 256)
P01_01_101 DONE
(45, 256)
P01_01_102 DONE
(54, 256)
P01_01_103 DONE
(120, 256)
P01_01_104 DONE
(46, 256)
P01_01_105 DONE
(45, 256)
P01_01_106 DONE
(70, 256)
P01_01_107 DONE
(72, 256)
P01_01_108 DONE
(59, 256)
P01_01_109 DONE
(58, 256)
P01_01_11 DONE
(127, 256)
P01_01_110 DONE
(197, 256)
P01_01_111 DONE
(108, 256)
P01_01_112 DONE
(136, 256)
P01_01_113 DONE
(107, 256)
P01_01_114 DONE
(75, 256)
P01_01_115 DONE
(76, 256)
P01_01_116 DONE
(56, 256)
P01_01_117 DONE
(54, 256)
P01_01_118 DONE
(399, 256)
P01_01_12 DONE
(67, 256)
P01_01_120 DONE
(57, 256)
P01_01_121 DONE
(63, 256)
P01_01_122 DONE
(97, 256)
P01_01_123 DONE
(46, 256)
P01_01_124 DONE
(66, 256)
P01_01_125 DONE
(232, 256)
P01_01_126 DONE
(73, 256

4it [03:20, 50.20s/it] 

P01_01_189 DONE
(87, 256)


100

In [ ]:
with open('p01_01_chunk_1_features.pkl', 'wb') as f:
  pickle.dump(output, f)

In [48]:
# features1 = features_and_meta['features']

# import cProfile

# def numpy_concat(x):
#     np.concatenate(x)

# cProfile.run('numpy_concat(features1)')

# allow = np.concatenate(features_and_meta['features'])

allow = np.concatenate(feature_writer.features)

allow.shape

(21794, 256)

In [11]:

features0 = features_and_meta['metadata']

f_1 = features0[0:8]

start = f_1[7]
start
# rgb_train['P01_01_1'][1], 
allow[]

'195'